<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

## Round 2, as TL
## THIS IS A DIFFERENT ASSIGNMENT
### Instructions are in the solution notebook

In [ ]:
import pandas as pd
import numpy as np

column_headers = ['symboling', 'normalized-losses', 'make', 'fuel-type', 
                  'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 
                  'engine-location', 'wheel-base', 'length', 'width', 'height', 
                  'curb-weight', 'engine-type', 'num-of-cylinders', 
                  'engine-size', 'fuel-system', 'bore', 'stroke', 
                  'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 
                  'highway-mpg', 'price']

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data', 
                 header=None,
                 names=column_headers,
                 na_values='?')
print(df.shape)
df.head(2)

(205, 26)


,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0


In [ ]:
# df.sample 10 cars with random seed 30

sample_10_cars = df.sample(n=10, random_state=30)
sample_10_cars.head(2)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
121,1,154.0,plymouth,gas,std,four,sedan,fwd,front,93.7,167.3,63.8,50.8,1989,ohc,four,90,2bbl,2.97,3.23,9.4,68.0,5500.0,31,38,6692.0
176,-1,65.0,toyota,gas,std,four,sedan,fwd,front,102.4,175.6,66.5,54.9,2414,ohc,four,122,mpfi,3.31,3.54,8.7,92.0,4200.0,27,32,10898.0


In [ ]:
from scipy.stats import ttest_1samp

ttest_1samp(sample_10_cars['highway-mpg'], df['highway-mpg'].mean(), nan_policy='omit')

Ttest_1sampResult(statistic=0.8912422391619923, pvalue=0.39600040771093326)

The salesman says the cars he sells typically have a fuel efficiency of about 35 miles per gallon on the highway. You want to verify his claim but can only test 10 cars. Using your sample of 10, test his claim and report your results.

In [ ]:
ttest_1samp(sample_10_cars['highway-mpg'], 35, nan_policy='omit')

# ...nice guy

Ttest_1sampResult(statistic=-0.9570244044334747, pvalue=0.36355472283248624)

In [ ]:
# now with 100-car sample

sample_100_cars = df.sample(100, random_state=30)

ttest_1samp(sample_100_cars['highway-mpg'], 35, nan_policy='omit')

Ttest_1sampResult(statistic=-6.5185242467032305, pvalue=3.003137417468754e-09)

In [ ]:
# yeah the average is 30

print(sample_100_cars['highway-mpg'].mean())

30.64


## Part 2 Congressional votes data

### 1) Load + clean

The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - *there are missing values!*

- Read the dataset in from UCI, you'll need to provide a list of column headers
- Encode "yes" votes as 1 and "no" votes as 0. (You can use `df.replace()` to swap out these values)
- Use dataframe filtering to split the dataframe into two new dataframes based on party. Hold all republicans in one dataframe and all democrats in the other. These will be our two different "samples."


In [ ]:
# Grab the file from UCI:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-07-14 00:00:07--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2020-07-14 00:00:08 (287 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [ ]:
column_headers = ['party', 'handicapped-infants', 'water-project', 
'budget', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups', 
'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration', 
'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free', 
'south-africa']

In [ ]:
df = pd.read_csv('house-votes-84.data', 
                 header=None,
                 names=column_headers,
                 na_values='?')

df = df.replace({'n': 0, 'y': 1})

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [ ]:
df.shape

(435, 17)

In [ ]:
# Split into republicans and democrats.
# These will be our two samples for t testing

reps = df[df['party'] == 'republican']
dems = df[df['party'] == 'democrat']

print(reps.shape)
print(dems.shape)
reps.head(2)

(168, 17)
(267, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN


In [ ]:
rep_list = []

reps_numerical = reps.drop('party', axis=1)


for issue in reps_numerical:
  rep_list.append(reps[issue].sum() / len(reps_numerical))

In [ ]:
dem_list = []

dems_numerical = dems.drop('party', axis=1)


for issue in dems_numerical:
  dem_list.append(dems[issue].sum() / len(dems_numerical))

In [ ]:
support_comparisons = pd.DataFrame({'reps': rep_list, 'dems': dem_list}, index=column_headers[1:])

support_comparisons['rep > dem by'] = support_comparisons['reps'] - support_comparisons['dems']

In [ ]:
support_comparisons.head(20)

,reps,dems,rep > dem by
handicapped-infants,0.184524,0.584270,-0.399746
water-project,0.446429,0.449438,-0.003010
budget,0.130952,0.865169,-0.734216
physician-fee-freeze,0.970238,0.052434,0.917804
el-salvador-aid,0.934524,0.205993,0.728531
religious-groups,0.886905,0.460674,0.426231
anti-satellite-ban,0.232143,0.749064,-0.516921
aid-to-contras,0.142857,0.816479,-0.673622
mx-missile,0.113095,0.704120,-0.591025
immigration,0.547619,0.464419,0.083200


In [ ]:
from scipy.stats import ttest_ind

In [ ]:
# Positive t stat --> rep support > dem support
# reject

ttest_ind(reps['physician-fee-freeze'], dems['physician-fee-freeze'], nan_policy='omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

In [ ]:
# Negative t stat --> rep support < dem support
# reject

ttest_ind(reps['budget'], dems['budget'], nan_policy='omit')

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

In [ ]:
# p value > 0.05
# fail to reject at any reasonable alpha level

ttest_ind(reps['water-project'], dems['water-project'], nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

### Stretch #1 automate the t tests
I already kind of automated the first part but here we go

In [ ]:
dems.columns

Index(['party', 'handicapped-infants', 'water-project', 'budget',
       'physician-fee-freeze', 'el-salvador-aid', 'religious-groups',
       'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration',
       'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free',
       'south-africa'],
      dtype='object')

In [ ]:
reps.columns

Index(['party', 'handicapped-infants', 'water-project', 'budget',
       'physician-fee-freeze', 'el-salvador-aid', 'religious-groups',
       'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration',
       'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free',
       'south-africa'],
      dtype='object')

In [ ]:
# Navroz was trying to use a zip object. Zips along 2 iterables until 1 runs out, making tuples...?
# 

#zipped = zip(dems,reps)

In [ ]:
from scipy import stats

dcols = dems.columns.tolist()
rcols = reps.columns.tolist()

def ttest(dems,reps):
  for col in dcols[1:]:
    print(stats.ttest_ind(dems[col], reps[col], nan_policy='omit'))
  return 

In [ ]:
ttest(dems,reps)

Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)
Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)
Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)
Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)
Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68)
Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)
Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31)
Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)
Ttest_indResult(statistic=16.437503268542994, pvalue=5.03079265310811e-47)
Ttest_indResult(statistic=-1.7359117329695164, pvalue=0.08330248490425066)
Ttest_indResult(statistic=8.293603989407588, pvalue=1.5759322301054064e-15)
Ttest_indResult(statistic=-20.500685724563073, pvalue=1.8834203990450192e-64)
Ttest_indResult(statistic=-13.51064251060933, pvalue=1.2278581709672758e-34)
Ttest_indRes

In [ ]:
dems_list = dems.columns.tolist()[1:]
reps_list = reps.columns.tolist()

In [ ]:
for col in dems_list:
  print('\n', col.upper())
  print(ttest_ind(dems[col], reps[col], nan_policy='omit'))


 HANDICAPPED-INFANTS
Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)

 WATER-PROJECT
Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

 BUDGET
Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

 PHYSICIAN-FEE-FREEZE
Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)

 EL-SALVADOR-AID
Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68)

 RELIGIOUS-GROUPS
Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)

 ANTI-SATELLITE-BAN
Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31)

 AID-TO-CONTRAS
Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)

 MX-MISSILE
Ttest_indResult(statistic=16.437503268542994, pvalue=5.03079265310811e-47)

 IMMIGRATION
Ttest_indResult(statistic=-1.7359117329695164, pvalue=0.08330248490425066)

 SYNFUELS
Ttest_indResult(statistic=8.293603989407588, pvalue=1.575932

## Stretch 2: t test by hand




### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

### Be sure to check your work using Scipy!

Null Hypothesis: Democrat support for the handicapped-infants bill is 50%

Alternative Hypothesis: Democrat support is not 50%.

Confidence Level: 95%

![1-sample t-stat calculation](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png)

In [ ]:
sample = dems['handicapped-infants'].copy()
sample = sample.dropna()

In [ ]:


x_bar = sample.sum() / len(sample)
mu = 0.50
s = np.std(sample)
n = len(sample)

In [ ]:
denom = s / np.sqrt(n)

In [ ]:
num = x_bar - mu

In [ ]:
t = num / denom
print('population mean', mu)
print('sample mean', x_bar)
print('sample std dev', s)
print('t statistic ', t)
print('sample size', n)

# according to table, p < 0.001 we reject 0.5 support

population mean 0.5
sample mean 0.6046511627906976
sample std dev 0.48892548934020097
t statistic  3.4380424294366003
sample size 258


In [ ]:
# If you didn't want to use table...

# Be sure to use two-sided
p = stats.t.sf(np.abs(t), n-1)*2
print("p-value:", p)

p-value: 0.000683349852026897


In [ ]:
# Check work with scipy
ttest_1samp(sample, 0.5)

Ttest_1sampResult(statistic=3.431373087696574, pvalue=0.0006996123171673836)

# BEGINNING OF OLD ASSIGNMENT

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-07-12 19:28:16--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.3’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2020-07-12 19:28:17 (137 KB/s) - ‘house-votes-84.data.3’ saved [18171/18171]



In [ ]:
column_headers = ['party', 'handicapped-infants', 'water-project', 
'budget', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups', 
'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration', 
'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free', 
'south-africa']

df = pd.read_csv('house-votes-84.data', 
                 header=None, 
                 names=column_headers, 
                 na_values='?')
                 
df = df.replace({'y': 1, 'n': 0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [ ]:
# You can tell whether the support is "greater than" or "less than" based on the sign of the t statistic.

In [ ]:
rep = df[df['party']=='republican']
rep.head()


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


SyntaxError: ignored

In [ ]:
dem = df[df['party']=='democrat']
dem.head()

In [ ]:
for column in rep:
  if column != 'party':
    print(rep[column].mean())

In [ ]:
for column in dem:
  if column != 'party':
    print(dem[column].mean())

In [ ]:
# try physician-fee-freeze for rep > dem
# Null hypothesis: no difference in support between the two parties.
print(rep['physician-fee-freeze'].mean())
print(dem['physician-fee-freeze'].mean())

In [ ]:
ttest_ind(rep['physician-fee-freeze'], dem['physician-fee-freeze'], nan_policy='omit')

# Due to a p-value of way (way) less than 0.01, we reject the null hypothesis
# and conclude that there is a significant difference in support between
# Republicans and Democrats. Republicans support this bill more, as evidenced 
# by the positive t-stat and visually examining the mean support levels above.
# Also note that a one-sided t-test would be easier to reject the null, so we
# have effectively given ourselves a 99.5% confidence level that republican
# support is greater than democrat support.

In [ ]:
#

In [ ]:
# try anti-satellite-ban for dem > rep
# Null hypothesis: no difference in support between the two parties.
print(rep['anti-satellite-ban'].mean())
print(dem['anti-satellite-ban'].mean())

In [ ]:
ttest_ind(rep['anti-satellite-ban'], dem['anti-satellite-ban'], nan_policy='omit')

# Due to p-value < 0.01 and a negative t-stat, 
# we reject the null hypothesis that there is no difference
# in support and conclude that democrat support is higher than republican support.
# 

In [ ]:
#

In [ ]:
## try water-project for dem = rep
# Null hypothesis: no difference in support between the two parties.
print(rep['water-project'].mean())
print(dem['water-project'].mean())

In [ ]:
ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')

# Due to p-value of ~0.92 and a t-stat of ~0.089, 
# we fail to reject the null hypothesis that there is no difference
# in support.